In [1]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max.columns', 100)
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from calendar import monthrange 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
import wikipedia
import random

In [19]:
pd.set_option('display.width', 1000) 
pd.set_option ('display.max_columns' , 1000)
pd.set_option("display.max_rows", 1000)


In [57]:
train_data = pd.read_csv('Data for comp/flight_delays_train.csv')
y_train = train_data['dep_delayed_15min']
x_test = pd.read_csv('Data for comp/flight_delays_test.csv')
train_data['dep_delayed_15min'] = train_data['dep_delayed_15min'].map({'N':0, 'Y':1})
x_train = train_data.drop(columns=['dep_delayed_15min'])

In [4]:
x_train = pd.concat([x_train, x_test], axis = 0)

In [5]:
x_train.index = np.arange(x_train.shape[0])

In [6]:
x_train.insert(loc = 0, column = 'Initial Index', value = pd.Series(np.arange(0,200000)))

In [7]:
Air_Ports_Codes_Names = pd.read_excel('Data for comp/Airports .xlsx')
Top_100_airports = pd.read_excel('Data for comp/Top Airports.xlsx')
Climate_features = pd.read_excel('Data for comp/Climate features .xlsx')
Air_line_codes = pd.read_excel('Data for comp/Airlines codes .xlsx')
Top_By_passengers_carried = pd.read_excel('Data for comp/Top By passengers carried .xlsx')
Top_By_fleet_size = pd.read_excel('Data for comp/Top By fleet size.xlsx')

In [8]:
Population_of_cities = pd.read_excel('Data for comp/Population .xlsx')

In [9]:
Population_of_cities.drop(columns=['Unnamed: 0'],inplace= True)

In [10]:
len(set(Population_of_cities['Geographic Area'])), Population_of_cities.shape[0]

(887, 887)

In [34]:
for i in np.arange(810, 887):
    Location = Population_of_cities.iloc[i, 0]
    Population = find_num_population(Location)
    Population_of_cities.iloc[i,1] = Population

In [35]:
Population_of_cities[Population_of_cities['Population'].isna()]

,Geographic Area,Population
809,"Adak, Alaska",NaN
817,"Christiansted, U.S. Virgin Islands",NaN
818,"Cincinnati, Kentucky",NaN
840,"Islip, New York",NaN
847,"Key West, Florida",NaN
849,"Kinston, North Carolina",NaN
862,"Monterey, California",NaN
868,"Newburgh, New York",NaN
876,"Sitka, Alaska",NaN


In [36]:
Numbers_ = [226,2626,301394,330914, 24843, 20398,28352,28866,8640]
for i in Population_of_cities[Population_of_cities['Population'].isna()].index:
    pos = 0
    Population_of_cities.loc[i, 'Population'] = Numbers_[pos]
    pos += 1

In [37]:
States_Time_zones = pd.read_excel('Data for comp/Time Zones.xlsx')
Time_zones = pd.read_excel('Data for comp/Differences between zones.xlsx')
States_Time_zones['State'] = States_Time_zones['State'].apply(lambda x: x.split('(')[0])
States_Time_zones['Time Zone'] = States_Time_zones['Time Zone'].apply(lambda x: x.split('(')[1][:-1])

In [38]:
Codes = Air_line_codes['Passenger carrier:'].apply(lambda x: x.split(' ')[len(x.split(' ')) - 1])
Codes = Codes.apply(lambda x: x[1:len(x) - 1])
Air_lines = Air_line_codes['Passenger carrier:'].apply(lambda x: ' '.join(x.split(' ')[:len(x.split(' '))-1]))
Air_line_codes = pd.DataFrame({'Air line': Air_lines, 'Code':Codes})

In [39]:
my_dict = {'N':0, 'Y':1}
y_train = y_train.map(my_dict)


In [40]:
def fill_nan(table):
    for col in table.columns:
        if table[col].dtype == 'int64' or table[col].dtype == 'float64':
            table[col] = table[col].fillna(table[col].mean())
    return table 

In [41]:
def mean_target_enc1(train_df, y_train, valid_df, skf):
    import warnings
    warnings.filterwarnings('ignore')
    
    glob_mean = y_train.mean()
    train_df = pd.concat([train_df, pd.Series(y_train, name='y')], axis=1)
    new_train_df = train_df.copy()
    
    cat_features = train_df.columns[train_df.dtypes == 'object'].tolist()    

    for col in cat_features:
        new_train_df[col + '_mean_target'] = np.nan

    for train_idx, valid_idx in skf.split(train_df, y_train):
        train_df_cv, valid_df_cv = train_df.iloc[train_idx, :], train_df.iloc[valid_idx, :]

        for col in cat_features:
            
            means = valid_df_cv[col].map(train_df_cv.groupby(col)['y'].mean())
            valid_df_cv[col + '_mean_target'] = means.fillna(glob_mean)
            
        new_train_df.iloc[valid_idx] = valid_df_cv
    
    for col in cat_features:
        means = valid_df[col].map(train_df.groupby(col)['y'].mean())
        valid_df[col + '_mean_target'] = means.fillna(glob_mean)
    
    return new_train_df, valid_df

In [42]:
def mean_target_enc_train(train_df, y_train, skf):
    import warnings
    warnings.filterwarnings('ignore')
    
    glob_mean = y_train.mean()
    train_df = pd.concat([train_df, pd.Series(y_train, name='y')], axis=1)
    new_train_df = train_df.copy()
    
    cat_features = train_df.columns[train_df.dtypes == 'object'].tolist()    

    for col in cat_features:
        new_train_df[col + '_mean_target'] = np.nan

    for train_idx, valid_idx in skf.split(train_df, y_train):
        train_df_cv, valid_df_cv = train_df.iloc[train_idx, :], train_df.iloc[valid_idx, :]

        for col in cat_features:
            
            means = valid_df_cv[col].map(train_df_cv.groupby(col)['y'].mean())
            valid_df_cv[col + '_mean_target'] = means.fillna(glob_mean)
            
        new_train_df.iloc[valid_idx] = valid_df_cv
    new_train_df.drop(cat_features + ["y"], axis=1, inplace=True)
    return new_train_df

In [43]:
Month_Seasons = {'1':'Winter',
                 '2':'Winter',
                 '3':'Spring', 
                 '4':'Spring', 
                 '5':'Spring', 
                 '6':'Summer', 
                 '7':'Summer', 
                 '8':'Summer', 
                 '9':'Fall', 
                 '10':'Fall', 
                 '11':'Fall', 
                 '12':'Winter'}

In [44]:
def extract_name_city(Pandas_Series):
    Series_of_cities = Pandas_Series.apply(lambda x: x.split(',')).apply(lambda x: x[0])
    Series_of_States  = Pandas_Series.apply(lambda x: x.split(',')).apply(lambda x: x[1])
    list_of_cities = []
    for city in Series_of_cities:
        my_list = city.split(' ')
        if 'city' in my_list:
            my_list.remove('city')
        if 'town'in my_list:
            my_list.remove('town')
        if 'village' in my_list:
            my_list.remove('village')
        if 'CDP' in my_list:
            my_list.remove('CDP')
        if 'urban' in my_list:
            my_list.remove('urban')
        if 'county' in my_list:
            my_list.remove('county')
        if '(Ventura)' in my_list:
            my_list.remove('(Ventura)')
        if 'municipality' in my_list:
            my_list.remove('municipality')
        if '(balance)' in my_list:
            my_list.remove('(balance)')
        if my_list[len(my_list)-1] == 'government':
            my_list = my_list[0]
        list_of_cities.append(my_list)
    return pd.Series(list_of_cities).map(lambda x: ' '.join(x)) + Series_of_States

In [45]:
def all_duplicates(list_):
    duplicates = []
    set_list = set(list_)
    dict_elements = {}
    for elem in list_:
        dict_elements[elem] = dict_elements.get(elem, 0) + 1
        if dict_elements[elem] > 1:
            duplicates.append(elem)
    return list(set(duplicates))

In [46]:
def is_it_holyday (date): 
    if date in ['1-1', '1-18','5-31','7-5','9-6','11-11','11-25','12-24','12-31']:
        return 'Holyday'
    else:
        return 'Not Holyday'

In [47]:
def is_week_end (num_day):
    num_day = str(num_day)
    if num_day == '6' or num_day == '7':
        return 'weekend'
    else:
        return ' not weekend'

In [48]:
def part_of_day (time):
    if 400 < time <= 1200:
        return 'Morning'
    if 1200 < time <= 1700:
        return 'Afternoon'
    if 1700 < time <= 2100:
        return 'Evening '
    if 2100 < time or time <= 400:
        return 'Night'

In [49]:
def part_of_day1 (time):
    if 0 < time <= 1200:
        return 'Morning'
    if 1200 < time <= 2400:
        return 'Afternoon'


In [50]:
def part_of_month_ (date_data, num_parts = 4):
    date_data = date_data.copy()
    zero_series = pd.Series(np.zeros(date_data.shape[0]))
    separated_data = date_data.apply(lambda x: x.split('-'))
    for index_date in np.arange(date_data.shape[0]):
        intervals = np.arange(1, num_parts+ 1) * (monthrange(2017,
                                                            int(separated_data[index_date][0]))[1])/num_parts
        for i in range(num_parts):
            if int(separated_data[index_date][1]) <= intervals[i]:
                zero_series[index_date] = i + 1
                break
    return zero_series

In [51]:
def time_zones_shifts (data):
    Time_zones = data.iloc[:,0].tolist()
    Times = data.iloc[:,1].tolist()
    dict_times_zones = dict()
    dict_time_shifts = dict()
    time_zone_combination = [*itertools.product(Time_zones, repeat=2)]
    for i in range(len(Time_zones)):
        dict_times_zones[Time_zones[i]] = dict_times_zones.get(Time_zones[i], Times[i])
    for i in range(len(time_zone_combination)):
        x_dest = time_zone_combination[i][0]
        y_dest = time_zone_combination[i][1]
        shift = dict_times_zones[y_dest] - dict_times_zones[x_dest]
        dict_time_shifts[x_dest+'-'+y_dest] = dict_time_shifts.get(x_dest+'-'+y_dest, shift) 
    return dict_time_shifts    

In [52]:
def sep_hours_minutes (data):
    def distinct_hours (time):
        if len(str(time)) == 4:
            return str(time)[:2]
        elif len(str(time)) == 3:
            return str(time)[0]
        else:
            return '0'
    def distinct_minutes (time):
        if len(str(time)) == 4:
            return str(time)[2:4]
        elif len(str(time)) == 3:
            return str(time)[1:3]
        else:
            return str(time)
    Series_of_hours = data.apply(distinct_hours)
    Series_of_minutes = data.apply(distinct_minutes)
    return Series_of_hours +'.'+Series_of_minutes

In [53]:
def sum_time(Time_series1, Time_series2):
    Sum_minutes = Time_series1.apply(lambda x: int(x.split('.')[1])) + \
    Time_series2.apply(lambda x: int(x.split('.')[1]))
    Sum_hours = Time_series1.apply(lambda x: int(x.split('.')[0])) + \
    Time_series2.apply(lambda x: int(x.split('.')[0]))
    Residual_minutes = (Sum_minutes%60).apply(str)
    Residual_hours = Sum_minutes//60
    Sum_hours += Residual_hours
    for i in Sum_hours.index:
        if Sum_hours[i] >= 24:
            Sum_hours[i] %=24
    for i in Residual_minutes.index:
        if int(Residual_minutes[i]) < 0:
            Residual_minutes[i] = '0'+ Residual_minutes[i]
    return Sum_hours.apply(str), Residual_minutes

In [54]:
def switch_to_time (number):
    num_minutes = int(number * 60)
    hours = num_minutes//60
    minutes = num_minutes%60
    if minutes < 10:
         minutes = '0'+str(minutes)
    return str(hours) + '.' + str(minutes)

In [55]:
def distinct_hours (time):
        if len(str(time)) == 4:
            return int(str(time)[:2])
        elif len(str(time)) == 3:
            return int(str(time)[0])
        else:
            return 0

In [56]:
def distinct_minutes (time):
        if len(str(time)) == 4:
            return str(time)[2:4]
        elif len(str(time)) == 3:
            return str(time)[1:3]
        else:
            return str(time)

In [57]:
def sum_colums_df (df):
    Columns = df.columns.tolist()
    Base_col = pd.Series(np.zeros(df.shape[0]))
    for col in Columns:
        Base_col += df[col]
    return Base_col 

In [58]:
def find_num_population(Name):
    def extract_int(list):
        list_int = []
        for i in list:
            try:
                list_int.append(int(i))
            except:
                pass
        return list_int
    Name = Name.replace(', ', ',')
    try:
        import wikipedia
        wikipedia_page = wikipedia.page(Name)
        list_words = [*map(lambda x: x.lower(),wikipedia_page.summary.split(' '))]
        loc = 0
        for word in list_words:
            if word == 'population':
                break
            else:
                loc += 1
        sub_list = list_words[loc: loc+20]
        word_list = []
        new_sub_list = []
        for word_ in sub_list:
            word_ = word_.replace('.\n\nthe', '').rstrip(',').rstrip(':').rstrip(';').rstrip('.')
            new_sub_list.append(word_)
            if ',' in word_:
                word_list.append(word_)
        if word_list != []:
            number = word_list[0].split(',')
            Population_region = int(number[0] + number[1])
            return Population_region
        else:
            ints = extract_int(new_sub_list)
            return ints[0]
    except:
        return np.NaN

In [59]:
def extract_geo_position (Location):
    try:
        page = wikipedia.page(Location)
        coordinates = [*map(lambda x: float(round(x, 4)), page.coordinates)]
        return coordinates
    except:
        try:
            page = wikipedia.page(wikipedia.search(Location)[1])
            coordinates = [*map(lambda x: float(round(x, 4)), page.coordinates)]
            return coordinates

        except:
            return [np.NaN, np.NaN]

In [60]:
def find_acres (Name):
    def extract_int(list):
        list_int = []
        for i in list:
            try:
                if ',' in i:
                    i_list = i.split(',')
                    i = i_list[0] + i_list[1]
                list_int.append(int(i))
            except:
                pass
        return list_int
    try:
        wikipedia_page = wikipedia.page(Name)
        list_words = [*map(lambda x: x.lower(),wikipedia_page.content.split(' '))]
        pos_acres = []
        loc = 0
        for word in list_words:
            word = word.replace('.\n\nthe', '').rstrip(',').rstrip(':').rstrip(';').rstrip('.')
            if word == 'acres':
                pos_acres.append(loc)
            else:
                loc += 1
        if len(pos_acres) == 0:
            return np.NaN
        for position in pos_acres:
            words_before_acre = list_words[position - 10:position + 1]
            if 'covers' in words_before_acre:
                all_ints = extract_int(words_before_acre)
                if len(all_ints) == 1:
                    num_arces = all_ints[0]
                    break
                else:
                    num_arces = all_ints[-1]
                    break
        return num_arces
    except:
        return np.NaN


In [61]:
def bootstrap_esimation (data, num_samples, num_selections):
    num_data_sumples = data.shape[0]
    num_samples = int(num_data_sumples * num_samples)
    dict_values = dict()
    for key in set(data.values):
        dict_values[key] = dict_values.get(key, [0] * num_selections)
    for i in np.arange(num_selections):
        index_selection = random.sample(range(0,num_data_sumples), num_samples)
        sample = data[index_selection]
        sample_values_share = sample.value_counts(normalize=True)
        for value in sample_values_share.index:
            dict_values[value][i] = sample_values_share[value]
    data_value_shares = pd.DataFrame(dict_values).mean()
    return data_value_shares

In [62]:
Population_of_cities['Geographic Area'] = extract_name_city(Population_of_cities['Geographic Area'])
Population_of_cities.rename(columns={'Geographic Area':'City'},inplace= True)

In [63]:
Air_Ports_Codes_Names['City'] = Air_Ports_Codes_Names['City'] + ' ' + Air_Ports_Codes_Names['State']

In [64]:
len(set(Population_of_cities['City'])), Population_of_cities.shape[0]

(887, 887)

In [65]:
New_xtrain = x_train.copy()

In [66]:
New_xtrain = New_xtrain.merge(Air_Ports_Codes_Names, left_on= 'Origin', right_on= 'IATA')
New_xtrain = New_xtrain.merge(Air_Ports_Codes_Names, left_on= 'Dest', right_on= 'IATA')
New_xtrain.rename(columns={'City_x':'City'}, inplace = True)
New_xtrain = New_xtrain.merge(Population_of_cities, on = 'City', how = 'left')
New_xtrain.rename(columns={'City':'Origin_City', 'Population':'Origin_Population', 'City_y': 'City'}, inplace = True)
New_xtrain = New_xtrain.merge(Population_of_cities, on = 'City', how = 'left')
New_xtrain.rename(columns={'City':'Dest_City', 'Population':'Dest_Population'}, inplace = True)

In [67]:
Top_50_airports_list = Top_100_airports['Code'].tolist()
Is_origin_airport_top = New_xtrain['Origin'].apply(lambda x: int(x in Top_50_airports_list))
Is_dest_airport_top = New_xtrain['Dest'].apply(lambda x: int(x in Top_50_airports_list))
df_air_port_top = pd.DataFrame({'Origin airport in top': Is_origin_airport_top,
                               'Dest airport in top':Is_dest_airport_top})

In [68]:
Top_fleet_size = Top_By_fleet_size.merge(Air_line_codes, left_on='American Airlines', 
                                           right_on= 'Air line')['Code'].tolist()
Top_passengers_carried = Top_By_passengers_carried.merge(Air_line_codes, left_on='Airline', 
                                           right_on= 'Air line')['Code'].tolist()
DF_TOP_features = pd.DataFrame({'AL_in_Top_fleet_size': New_xtrain['UniqueCarrier'].apply(lambda x: int(x in Top_fleet_size)),
                   'AL_in_Top_passengers_carried': New_xtrain['UniqueCarrier'].apply(lambda x: int(x in Top_passengers_carried))})
                   

In [69]:
New_xtrain.head(1)

,Initial Index,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,Airport_x,Origin_City,State_x,IATA_x,Airport_y,Dest_City,State_y,IATA_y,Origin_Population,Dest_Population
0,0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,Hartsfield–Jackson Atlanta International Airport,Atlanta Georgia,Georgia,ATL,Dallas/Fort Worth International Airport,Dallas Texas,Texas,DFW,491670.0,1342479.0


In [70]:
New_xtrain = pd.concat([New_xtrain ,DF_TOP_features, df_air_port_top], axis = 1)

In [71]:
New_xtrain['Month'] = New_xtrain['Month'].apply(lambda x: x[2:])
New_xtrain['DayofMonth'] = New_xtrain['DayofMonth'].apply(lambda x: x[2:])
New_xtrain['DayOfWeek'] = New_xtrain['DayOfWeek'].apply(lambda x: x[2:])

In [72]:
New_xtrain.sort_values(by = 'Initial Index', ascending= True, inplace = True)

In [73]:
Time_data = New_xtrain[['Initial Index', 'Month','DayofMonth','DayOfWeek','DepTime','Distance','State_x','State_y']]

In [74]:
Time_data1 = Time_data.merge(States_Time_zones, left_on= 'State_x', right_on= 'State')
Time_data2 = Time_data1.merge(States_Time_zones, left_on= 'State_y', right_on= 'State')
Time_data2.drop(columns = ['State_x', 'State_y'], inplace = True)

In [75]:
dict_distances = time_zones_shifts(Time_zones)

In [76]:
Time_data2['Shift Time'] = (Time_data2['Time Zone_x'] + '-' + Time_data2['Time Zone_y']).map(dict_distances)
Time_data2.drop(columns=['Time Zone_x','Time Zone_y'], inplace= True)

In [77]:
New_xtrain.head(4)

,Initial Index,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,Airport_x,Origin_City,State_x,IATA_x,Airport_y,Dest_City,State_y,IATA_y,Origin_Population,Dest_Population,AL_in_Top_fleet_size,AL_in_Top_passengers_carried,Origin airport in top,Dest airport in top
0,0,8,21,7,1934,AA,ATL,DFW,732,Hartsfield–Jackson Atlanta International Airport,Atlanta Georgia,Georgia,ATL,Dallas/Fort Worth International Airport,Dallas Texas,Texas,DFW,491670.0,1342479.0,0,0,1,1
121276,1,4,20,3,1548,US,PIT,MCO,834,Pittsburgh International Airport,Pittsburgh Pennsylvania,Pennsylvania,PIT,Orlando International Airport,Orlando Florida,Florida,MCO,301494.0,281804.0,0,0,1,1
18325,2,9,2,5,1422,XE,RDU,CLE,416,Raleigh-Durham International Airport,Raleigh North Carolina,North Carolina,RDU,Cleveland-Hopkins International Airport,Cleveland Ohio,Ohio,CLE,465776.0,385252.0,0,0,1,1
12703,3,11,25,6,1015,OO,DEN,MEM,872,Denver International Airport,Denver Colorado,Colorado,DEN,Memphis International Airport,Memphis Tennessee,Tennessee,MEM,704961.0,650878.0,0,0,1,0


In [78]:
flight_time  = round((Time_data2['Distance']/561),2).apply(switch_to_time)

In [79]:
Hours = sum_time(sep_hours_minutes(Time_data2['DepTime']),flight_time)[0]
Minutes = sum_time(sep_hours_minutes(Time_data2['DepTime']),flight_time)[1]

In [80]:
for i in Minutes.index:
    if int(Minutes[i]) < 10:
        Minutes[i] = '0'+ Minutes[i]

In [81]:
dest_time = (Hours + Minutes).apply(int)

In [82]:
shifter_dest_time = (dest_time.apply(distinct_hours) + Time_data2['Shift Time'])

In [83]:
for i in range(shifter_dest_time.shape[0]):
     if shifter_dest_time[i] < 0:
        shifter_dest_time[i] = 24 + shifter_dest_time[0]

In [84]:
Time_data2

,Initial Index,Month,DayofMonth,DayOfWeek,DepTime,Distance,Shift Time
0,0,8,21,7,1934,732,-1
1,255,4,12,3,925,874,-1
2,356,11,7,2,1418,874,-1
3,442,4,22,5,1438,696,-1
4,950,11,29,2,716,732,-1
...,...,...,...,...,...,...,...
199995,113881,6,11,1,1338,865,1
199996,118563,3,27,2,1801,865,1
199997,150559,6,3,7,2028,865,1
199998,178501,8,14,2,1338,865,1


In [85]:
Time_data2['Season'] = Time_data2['Month'].map(Month_Seasons)
Time_data2['Is it holyday'] = (Time_data2['Month'] + '-'+  Time_data2['DayofMonth']).apply(is_it_holyday)
Time_data2['Is it weekend'] = Time_data2['DayOfWeek'].apply(is_week_end)
Time_data2['Part of month'] = part_of_month_(Time_data2['Month'] + '-' + Time_data2['DayofMonth'], 4)
Time_data2['Land time'] = (shifter_dest_time.apply(str) + dest_time.apply(distinct_minutes)).apply(int)
Time_data2['Land part of day'] = Time_data2['Land time'].apply(part_of_day)
Time_data2['_2_part_of_day'] = Time_data2['DepTime'].apply(part_of_day1)
Time_data2['_2_part_of_day_land'] = Time_data2['Land time'].apply(part_of_day1)

In [86]:
Time_data2.insert(loc = 5, column= 'Dep part of day', value= Time_data2['DepTime'].apply(part_of_day))

In [87]:
Time_data2.sort_values(by = 'Initial Index', ascending= True, inplace = True)

In [88]:
New_xtrain.drop(columns=['Month','DayofMonth','DayOfWeek', 'Distance','DepTime'], inplace= True)

In [89]:
New_xtrain = Time_data2.merge(New_xtrain, left_on= 'Initial Index', right_on= 'Initial Index')

In [90]:
for i in New_xtrain[New_xtrain['Land time'] > 2400].index:
    Time = New_xtrain.loc[i, 'Land time']
    if 2400 < Time < 4300:
        Time_str = str(Time)
        Minutes = Time_str[2:]
        New_xtrain.loc[i, 'Land time'] = int(Minutes)
    if Time >= 4300:
        Time_str = str(Time)
        Minutes = Time_str[2:]
        New_xtrain.loc[i, 'Land time'] = int('23'+Minutes)
        

In [91]:
Climate_data = New_xtrain[['Initial Index',
                           'Season',
                           '_2_part_of_day',
                           'State_x']]

In [92]:
Climate_data = Climate_data.rename(columns = {'State_x':'State'})
Climate_data = (Climate_data.merge(Climate_features, on = 'State', how = 'left'))
Climate_data = fill_nan(Climate_data)

In [93]:
for i in np.arange(Climate_data.shape[0]):
    Season = Climate_data.loc[i,'Season']
    for col in Climate_data.columns[4:]:
        if Season not in col.split(' '):
            Climate_data.loc[i, col] = 0

In [94]:
C_f_temp = Climate_data.loc[:,'Spring - Avg ° C':'Winter - Avg ° C']
C_f_Millimetres = Climate_data.loc[:,'Spring - Milli­metres':'Winter - Milli­metres']
C_f_Total_SUN_Hours = Climate_data[['Spring SUN Total Hours',
                                              'Summer SUN Total Hours',
                                              'Fall SUN Total Hours',
                                              'Winter SUN Total Hours']]
C_f_SUN_Clear_Days = Climate_data[['Spring SUN Clear Days',
                                             'Summer SUN Clear Days', 
                                             'Fall SUN Clear Days', 
                                             'Winter SUN Clear Days']]
C_f_Humidity_Morning = Climate_data[['Spring Humidity Morn­ing',
                                               'Summer Humidity Morn­ing',
                                               'Fall Humidity Morn­ing',
                                               'Winter Humidity Morn­ing']]
C_f_Afternoon = Climate_data[['Spring Humidity After­noon',
                                        'Summer Humidity After­noon',
                                        'Fall Humidity After­noon',
                                        'Winter Humidity After­noon']]


In [95]:
List_cl_df = [C_f_temp,
              C_f_Millimetres,
              C_f_Total_SUN_Hours,
              C_f_SUN_Clear_Days,
              C_f_Humidity_Morning,
              C_f_Afternoon]
Name_features = ['Avg ° C Temp State_x',
                 'Prec Mill State_x',
                 'Sun hours State_x',
                 'Clear days State_x',
                 'Humidity Morning State_x',
                 'Humidity Afternoon State_x']
State_x_cl_df = pd.DataFrame()
index_name = 0
for df in List_cl_df:
    State_x_cl_df[Name_features[index_name]] = sum_colums_df(df)
    index_name += 1
State_x_cl_df['State_x'] = Climate_data['State']
State_x_cl_df['Part_of_day'] = Climate_data['_2_part_of_day']
for i in np.arange(State_x_cl_df.shape[0]):
    Part_of_day = State_x_cl_df.loc[i,'Part_of_day']
    for col in ['Humidity Morning State_x','Humidity Afternoon State_x']:
        if Part_of_day not in col.split(' '):
            State_x_cl_df.loc[i, col] = 0
State_x_cl_df['Humidity'] = State_x_cl_df['Humidity Morning State_x'] + \
State_x_cl_df['Humidity Afternoon State_x']
State_x_cl_df.drop(columns = ['Humidity Morning State_x','Humidity Afternoon State_x','State_x', 'Part_of_day'], inplace = True)

In [96]:
State_x_cl_df

,Avg ° C Temp State_x,Prec Mill State_x,Sun hours State_x,Clear days State_x,Humidity
0,25.9,120.0,882.0,22.0,54.0
1,8.6,93.0,716.0,19.0,49.0
2,15.7,100.0,617.0,34.0,53.0
3,7.7,30.0,734.0,42.0,60.0
4,12.1,80.0,565.0,29.0,56.0
...,...,...,...,...,...
199995,27.3,69.0,935.0,41.0,82.0
199996,12.1,80.0,565.0,29.0,56.0
199997,-2.1,56.0,422.0,21.0,66.0
199998,-4.8,73.0,289.0,8.0,77.0


In [97]:
Climate_data1 = New_xtrain[['Initial Index',
                           'Season',
                           '_2_part_of_day_land',
                           'State_y']]
Climate_data1.rename(columns = {'State_y': 'State'}, inplace = True)
Climate_data1 = (Climate_data1.merge(Climate_features, on = 'State', how = 'left')).fillna(0)

In [98]:
for i in np.arange(Climate_data1.shape[0]):
    Season = Climate_data1.loc[i,'Season']
    for col in Climate_data1.columns[4:]:
        if Season not in col.split(' '):
            Climate_data1.loc[i, col] = 0

In [99]:
C_f_temp1 = Climate_data1.loc[:,'Spring - Avg ° C':'Winter - Avg ° C']
C_f_Millimetres1 = Climate_data1.loc[:,'Spring - Milli­metres':'Winter - Milli­metres']
C_f_Total_SUN_Hours1 = Climate_data1[['Spring SUN Total Hours',
                                              'Summer SUN Total Hours',
                                              'Fall SUN Total Hours',
                                              'Winter SUN Total Hours']]
C_f_SUN_Clear_Days1 = Climate_data1[['Spring SUN Clear Days',
                                             'Summer SUN Clear Days', 
                                             'Fall SUN Clear Days', 
                                             'Winter SUN Clear Days']]
C_f_Humidity_Morning1 = Climate_data1[['Spring Humidity Morn­ing',
                                               'Summer Humidity Morn­ing',
                                               'Fall Humidity Morn­ing',
                                               'Winter Humidity Morn­ing']]
C_f_Afternoon1 = Climate_data1[['Spring Humidity After­noon',
                                        'Summer Humidity After­noon',
                                        'Fall Humidity After­noon',
                                        'Winter Humidity After­noon']]


In [100]:
List_cl_df1 = [C_f_temp1,
              C_f_Millimetres1,
              C_f_Total_SUN_Hours1,
              C_f_SUN_Clear_Days1,
              C_f_Humidity_Morning1,
              C_f_Afternoon1]
Name_features1 = ['Avg ° C Temp State_y',
                 'Prec Mill State_y',
                 'Sun hours State_y',
                 'Clear days State_y',
                 'Humidity Morning State_y',
                 'Humidity Afternoon State_y']
State_y_cl_df = pd.DataFrame()
index_name = 0
for df in List_cl_df1:
    State_y_cl_df[Name_features1[index_name]] = sum_colums_df(df)
    index_name += 1
State_y_cl_df['State_y'] = Climate_data1['State']
State_y_cl_df['Part_of_day'] = Climate_data1['_2_part_of_day_land']
for i in np.arange(State_y_cl_df.shape[0]):
    Part_of_day = State_y_cl_df.loc[i,'Part_of_day']
    for col in ['Humidity Morning State_y','Humidity Afternoon State_y']:
        if Part_of_day not in col.split(' '):
            State_y_cl_df.loc[i, col] = 0
State_y_cl_df['Humidity State_y'] = State_y_cl_df['Humidity Morning State_y'] + \
State_y_cl_df['Humidity Afternoon State_y']
State_y_cl_df.drop(columns = ['Humidity Morning State_y','Humidity Afternoon State_y','State_y', 'Part_of_day'], inplace = True)

In [101]:
Agregate_climate_data = pd.concat([State_x_cl_df, State_y_cl_df], axis= 1)

In [102]:
New_xtrain = pd.concat([New_xtrain,Agregate_climate_data], axis= 1)

In [103]:
Final_x_train = New_xtrain.copy()

In [104]:
Final_x_train.drop(columns=['Initial Index',
                            'Shift Time',
                            '_2_part_of_day',
                            '_2_part_of_day_land',
                            'Airport_x',
                            'IATA_x',
                            'Airport_y','IATA_y'], inplace= True)

In [105]:
dict_data_types = {'Part of month': 'object',
                   'AL_in_Top_fleet_size':'object', 
                   'AL_in_Top_passengers_carried':'object',
                  'Origin airport in top':'object',
                  'Dest airport in top':'object'}

In [106]:
New_xtrain.drop(columns=['Initial Index',
                            'Shift Time',
                            '_2_part_of_day',
                            '_2_part_of_day_land',
                            'Airport_x',
                            'IATA_x',
                            'Airport_y','IATA_y'], inplace= True)

In [107]:
New_xtrain['Target'] = y_train

In [111]:
Copy_New_xtrain = New_xtrain.copy()

In [117]:
Copy_New_xtrain.to_csv('Extended all data')

In [118]:
Copy_New_xtrain

,Month,DayofMonth,DayOfWeek,DepTime,Dep part of day,Distance,Season,Is it holyday,Is it weekend,Part of month,Land time,Land part of day,UniqueCarrier,Origin,Dest,Origin_City,State_x,Dest_City,State_y,Origin_Population,Dest_Population,AL_in_Top_fleet_size,AL_in_Top_passengers_carried,Origin airport in top,Dest airport in top,Avg ° C Temp State_x,Prec Mill State_x,Sun hours State_x,Clear days State_x,Humidity,Avg ° C Temp State_y,Prec Mill State_y,Sun hours State_y,Clear days State_y,Humidity State_y,Target
0,8,21,7,1934,Evening,732,Summer,Not Holyday,weekend,3.0,1952,Evening,AA,ATL,DFW,Atlanta Georgia,Georgia,Dallas Texas,Texas,491670.0,1342479.0,0,0,1,1,25.9,120.0,882.0,22.0,54.0,27.3,69.0,935.0,41.0,44.0,0.0
1,4,20,3,1548,Afternoon,834,Spring,Not Holyday,not weekend,3.0,1717,Evening,US,PIT,MCO,Pittsburgh Pennsylvania,Pennsylvania,Orlando Florida,Florida,301494.0,281804.0,0,0,1,1,8.6,93.0,716.0,19.0,49.0,21.1,92.0,881.0,33.0,52.0,0.0
2,9,2,5,1422,Afternoon,416,Fall,Not Holyday,not weekend,1.0,1506,Afternoon,XE,RDU,CLE,Raleigh North Carolina,North Carolina,Cleveland Ohio,Ohio,465776.0,385252.0,0,0,1,1,15.7,100.0,617.0,34.0,53.0,11.6,75.0,499.0,24.0,56.0,0.0
3,11,25,6,1015,Morning,872,Fall,Holyday,weekend,4.0,1248,Afternoon,OO,DEN,MEM,Denver Colorado,Colorado,Memphis Tennessee,Tennessee,704961.0,650878.0,0,0,1,0,7.7,30.0,734.0,42.0,60.0,14.8,101.0,591.0,33.0,52.0,0.0
4,10,7,6,1828,Evening,423,Fall,Not Holyday,weekend,1.0,1913,Evening,WN,MDW,OMA,Chicago Illinois,Illinois,Omaha Nebraska,Nebraska,2711069.0,476271.0,1,1,1,0,12.1,80.0,565.0,29.0,56.0,9.7,41.0,610.0,34.0,52.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,6,5,2,852,Morning,187,Summer,Not Holyday,not weekend,1.0,911,Morning,WN,CRP,HOU,Corpus Christi Texas,Texas,Houston Texas,Texas,325568.0,2316750.0,1,1,0,1,27.3,69.0,935.0,41.0,82.0,27.3,69.0,935.0,41.0,82.0,NaN
199996,11,24,6,1446,Afternoon,1515,Fall,Not Holyday,weekend,4.0,1528,Afternoon,UA,ORD,LAS,Chicago Illinois,Illinois,Las Vegas Nevada,Nevada,2711069.0,635262.0,0,0,1,1,12.1,80.0,565.0,29.0,56.0,10.1,19.0,851.0,46.0,30.0,NaN
199997,1,30,2,1509,Afternoon,438,Winter,Not Holyday,not weekend,4.0,1555,Afternoon,OO,ORD,SGF,Chicago Illinois,Illinois,Springfield Missouri,Missouri,2711069.0,167116.0,0,0,1,0,-2.1,56.0,422.0,21.0,66.0,0.2,58.0,471.0,24.0,56.0,NaN
199998,1,5,5,804,Morning,761,Winter,Not Holyday,not weekend,1.0,925,Morning,DL,LGA,ATL,New York New York,New York,Atlanta Georgia,Georgia,8437478.0,491670.0,1,1,1,1,-4.8,73.0,289.0,8.0,77.0,8.8,116.0,554.0,25.0,81.0,NaN


In [76]:
New_xtrain = pd.read_csv('Data for comp/Extended all data acre climate.csv')
Air_Port_data = pd.read_csv('Data for comp/Fixes Airpord data.csv')
New_xtrain.drop(columns = ['Acre_x','Acre_y'], inplace = True)

In [77]:
Air_Port_data = Air_Port_data[['IATA','Acre']]

In [79]:
New_xtrain = New_xtrain.merge(Air_Port_data, left_on='Origin', right_on='IATA')
New_xtrain = New_xtrain.merge(Air_Port_data, left_on='Dest', right_on='IATA')

In [78]:
Air_Port_data.loc[140, 'Acre'] = 650
Air_Port_data.loc[218, 'Acre'] = 2800

In [80]:
New_xtrain.sort_values(by = 'Unnamed: 0', inplace= True)

In [81]:
New_xtrain.drop(columns = ['IATA_x','IATA_y'], inplace = True)

In [82]:
New_xtrain.index = np.arange(200000)

In [84]:
States_Population = pd.read_excel('Data for comp/States_population .xlsx')

In [54]:
Target = New_xtrain['Target']

In [85]:
New_xtrain['Internal flight'] =(New_xtrain['State_x'] + ','+ New_xtrain['State_y']).apply(lambda x: int(x.split(',')[0] == x.split(',')[1]))

In [612]:
New_xtrain.drop(columns=['Target'], inplace= True)

In [86]:
all_data = pd.concat([x_train, x_test], axis=0)

In [87]:
all_data['Flight'] = all_data['Origin'] +'-'+ all_data['Dest']

In [88]:
all_data['Carrier-Flight'] = all_data['Flight'] + '-'+ all_data['UniqueCarrier']

In [89]:
Flight_data = all_data[['UniqueCarrier', 'Flight', 'Carrier-Flight']]

In [90]:
Dict_UniqueCarrier = dict(Flight_data['UniqueCarrier'].value_counts(normalize = True))
Dict_Flight = dict(Flight_data['Flight'].value_counts(normalize = True))
Dict_Carrier_Flight = dict(Flight_data['Carrier-Flight'].value_counts(normalize = True))

In [91]:
New_xtrain['Share UniqueCarrier'] = New_xtrain['UniqueCarrier'].map(Dict_UniqueCarrier)
New_xtrain['Share Flight'] = (New_xtrain['Origin'] +'-'+ New_xtrain['Dest']).map(Dict_Flight)
New_xtrain['Share Carrier-Flight'] = ((New_xtrain['Origin'] +'-'+ New_xtrain['Dest'])+'-'+New_xtrain['UniqueCarrier']).map(Dict_Carrier_Flight)

In [92]:
New_xtrain = New_xtrain.merge(States_Population, left_on='State_x', right_on='State')
New_xtrain.drop(columns = ['State'], inplace = True)

In [93]:
New_xtrain.rename(columns={'July 2019 Estimate':'Population State_x'}, inplace= True)

In [94]:
New_xtrain = New_xtrain.merge(States_Population, left_on='State_y', right_on='State')
New_xtrain.drop(columns = ['State'], inplace = True)
New_xtrain.rename(columns={'July 2019 Estimate':'Population State_y'}, inplace= True)

In [97]:
New_xtrain.index = np.arange(200000)

In [95]:
New_xtrain.sort_values(by = 'Unnamed: 0', inplace= True)

In [98]:
New_xtrain.drop(columns=['Unnamed: 0','Unnamed: 0.1'],inplace= True)


In [68]:
New_xtrain = pd.concat([New_xtrain,Target ], axis = 1)

In [101]:
New_xtrain.to_csv('Final all data flight delays.csv')